## 连接Ansys功能
首先创建一个控制的实例，使用`Ansys.connect_to_proj()`函数

In [1]:
from power_toys.components.ansys import Ansys
# 填入对应的项目名和design名字，如果没有已经打开的项目，将新建
ansys = Ansys()
ansys.connect_to_proj('Sim_48_6_FullBridge','Maxwell3DDesign1',desktop_version="2022.1")
modeler = ansys.m3d.modeler


PyAEDT INFO: StdOut has been enabled
PyAEDT INFO: using existing logger.
PyAEDT INFO: Launching PyAEDT outside AEDT with CPython and PythonNET.
PyAEDT INFO: AEDT installation Path D:\Program Files\Ansys22\v221\Win64.
PyAEDT INFO: Launching AEDT with module PythonNET.
PyAEDT INFO: AEDT 2022.1 Started with process ID 24932.
PyAEDT INFO: pyaedt v0.6.81
PyAEDT INFO: Python version 3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]
PyAEDT INFO: AEDT 2022.1.0 Build Date 2021-11-22 22:18:25
PyAEDT INFO: Project Sim_48_6_FullBridge has been created.
PyAEDT INFO: Added design 'Maxwell3DDesign1' of type Maxwell 3D.
PyAEDT INFO: Aedt Objects initialized
PyAEDT INFO: Variable Manager initialized
PyAEDT INFO: Design Loaded
PyAEDT INFO: Materials Loaded
PyAEDT INFO: aedt file load time 0.001994609832763672


In [2]:
# 导入sm3文件
modeler.import_3d_cad("E:\\workspace\\AD\\11kW_OBC\\Experiment_DEMO\\ANF\\Transformer_better_rearrange_winding_side_winding.sm3")
# 旋转模型，为了跟AD的坐标系保持一致
modeler.rotate(modeler.object_list, 'XY')
# 删除介质层

modeler.delete([modeler.get_object_from_name('DIELECTRIC')])
modeler.fit_all()

PyAEDT INFO: Step file E:\workspace\AD\11kW_OBC\Experiment_DEMO\ANF\Transformer_better_rearrange_winding_side_winding.sm3 imported
PyAEDT INFO: Deleted 1 Objects: DIELECTRIC.


## 创建磁芯

In [5]:
ansys.import_ferrite()
anchor = modeler.get_object_from_name('ANCHOR_2')
center_x = (anchor.bounding_box[0]+anchor.bounding_box[3])/2
center_y = (anchor.bounding_box[1]+anchor.bounding_box[4])/2

ansys.m3d['_k'] = '600percent'
ansys.m3d['_r'] = '6.5mm'
ansys.m3d['_w'] = '11.5mm'


ansys.m3d.assign_material(modeler.object_list, 'Copper')
ansys.create_strip(position=[f"({center_x}mm)",f"{center_y}mm",'-1mm'],r="_r",k="_k",along='X',matname='ferrite_loss',name='left')
ansys.create_strip(position=[f"({center_x}mm)",f"{center_y}mm+36mm",'-1mm'],r="_r",k="_k",along='X',matname='ferrite_loss',name='right')

PyAEDT WARNING: Material ferrite_loss already exists. Renaming to ferrite_loss_QOBYKM


PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT INFO: Union of 3 objects has been executed.
PyAEDT INFO: Union of 3 objects has been executed.


'right'

In [6]:
ansys.create_I_plate(position=[f"({center_x}mm)-_r",f"{center_y}mm-_r",'2.5mm'],height='9mm',r="_r",k="_k",w='_w',matname='ferrite_loss')
ansys.create_I_plate(position=[f"({center_x}mm)-_r",f"{center_y}mm-_r",'-1.5mm'],height='-9mm',r="_r",k="_k",w='_w',matname='ferrite_loss')


In [7]:
def create_terminal(obj_name):
    terminal_tmp = ansys.m3d.modeler.get_object_from_name(obj_name)
    pos_tmp_top_raw = terminal_tmp.bounding_box[:3]
    pos_tmp_top_raw [2] += terminal_tmp.bounding_dimension[2]
    pos_tmp = [f"{x}mm" for x in pos_tmp_top_raw]
    dim_tmp = [f"{x}mm" for x in terminal_tmp.bounding_dimension[:2]]
    dim_tmp.append('50mm')
    input_tmp = modeler.create_box(position=pos_tmp,dimensions_list=dim_tmp)
    ansys.m3d.assign_material(input_tmp, 'Copper')
    return input_tmp

pri_input = create_terminal('W1_49')
pri_output = create_terminal('W1_65')
sec_input = create_terminal('W2_49')
sec_output = create_terminal('W2_60')

r1_input = create_terminal('W3_8')
r1_output = create_terminal('W3_9')
r2_input = create_terminal('W4_8')
r2_output = create_terminal('W4_9')


PyAEDT DEBUG: Exporting 1 objects


PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects


In [8]:
ansys.m3d.assign_current([pri_input.top_face_z],amplitude='15A',name='pri_in')
ansys.m3d.assign_current([pri_output.top_face_z],amplitude='15A',swap_direction=True,name='pri_out')
ansys.m3d.assign_current([sec_input.top_face_z],amplitude='15A',name='sec_in')
ansys.m3d.assign_current([sec_output.top_face_z],amplitude='15A',swap_direction=True,name='sec_out')

ansys.m3d.assign_current([r1_input.top_face_z],amplitude='15A',name='r1_in')
ansys.m3d.assign_current([r1_output.top_face_z],amplitude='15A',swap_direction=True,name='r1_out')
ansys.m3d.assign_current([r2_input.top_face_z],amplitude='15A',name='r2_in')
ansys.m3d.assign_current([r2_output.top_face_z],amplitude='15A',swap_direction=True,name='r2_out')

In [9]:
modeler.create_region([200,200,0,200,200,200])

In [8]:
ansys.m3d.create_setup(setupname='Setup_500kHz',Frequency = '500kHz',HasSweepSetup = False)

SetupName Setup_500kHz with 0 Sweeps

In [11]:
ansys.rename_winding(['W1'], 100, 'pri', 'winding')
ansys.rename_winding(['W2'], 100, 'sec', 'winding')

PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects


In [1]:
print(debug)

NameError: name 'debug' is not defined

In [ ]:
name_assigned = []
for obj in winding_list:
    layer = pos_z_list.index(obj.bounding_box[2])
    new_name = f"Layer_{layer}_1"

    if(new_name in name_assigned):
        new_name = f"Layer_{layer}_2"

    obj.name = new_name
    name_assigned.append(new_name)
    intrinsic = {"Freq": "500kHz", "Phase": "0deg"}
m3d.post.create_fieldplot_volume(
    winding_list,
    'Mag_J',
    m3d.existing_analysis_sweeps[0],
    intrinsic,
    plot_name="Mag_J"
)

PyAEDT DEBUG: Exporting 1 objects


PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT DEBUG: Exporting 1 objects
PyAEDT INFO: aedt file load time 0.6621396541595459


In [ ]:
for obj in m3d.modeler.get_objects_in_group("winding"):
    Fields = m3d.odesign.GetModule("FieldsReporter")
    # 计算所有欧姆Loss
    Fields.EnterQty("OhmicLoss")
    Fields.CalcOp("Smooth")
    Fields.EnterVol(obj)
    Fields.CalcOp('Integrate')
    Fields.AddNamedExpression(f"{obj}_loss", "Fields")
    


In [ ]:
winding_loss_dict = {}
for obj in m3d.modeler.get_objects_in_group("winding"):
    winding_loss_dict[obj] = Ansys.get_scalar_value(m3d,f"{obj}_loss")

PyAEDT INFO: Solution Data Correctly Loaded.
PyAEDT INFO: Solution Data Correctly Loaded.
PyAEDT INFO: Solution Data Correctly Loaded.
PyAEDT INFO: Solution Data Correctly Loaded.
PyAEDT INFO: Solution Data Correctly Loaded.
PyAEDT INFO: Solution Data Correctly Loaded.
PyAEDT INFO: Solution Data Correctly Loaded.
PyAEDT INFO: Solution Data Correctly Loaded.
PyAEDT INFO: Solution Data Correctly Loaded.
PyAEDT INFO: Solution Data Correctly Loaded.
PyAEDT INFO: Solution Data Correctly Loaded.
PyAEDT INFO: Solution Data Correctly Loaded.
PyAEDT INFO: Solution Data Correctly Loaded.
PyAEDT INFO: Solution Data Correctly Loaded.
PyAEDT INFO: Solution Data Correctly Loaded.
PyAEDT INFO: Solution Data Correctly Loaded.


## 在ansys中添加绘图
这里以绘制bottom_plate中的B场幅值为例

In [ ]:
# # 在volume上添加B场幅值
Ansys.create_field_plot_volume(m3d, "bottom_plate", 'Mag_J', 'MagB in bottom plate')

更高阶的功能可以直接通过操作pyaedt的API来实现如下所示

In [ ]:
# quantity_name2 = "Mag_B"
# setup_name = "Setup1"
# intrinsic = {"Freq": "1MHz", "Phase": "0deg"}
# vollist = ["Top_plate_bottom"]

# # 在某一个面上绘制B场幅值
# plot2 = m3d.post.create_fieldplot_surface(m3d.modeler["Top_plate"].faces[1].id, 'Mag_B', m3d.existing_analysis_sweeps[0], intrinsic)
# # 导出图片
# m3d.post.export_field_jpg(
#     'E:\\Desktop\\Power_toys\\Power_toys\\prova2.jpg',
#     plot2.name,
#     plot2.plotFolder,
#     orientation='top',
#     display_wireframe=False,
# )
# # 导出数据到文件
# m3d.post.export_field_file_on_grid(
#     quantity_name="Mag_B",
#     solution="Setup1 : LastAdaptive",
#     filename='E:\\Desktop\\Power_toys\\Power_toys\\Mag.csv',
#     grid_start=[6,6,-0.2],
#     grid_stop=[6, 7, -0.2],
#     grid_step=[0.1, 0.1, 0],
#     intrinsics="1MHz",
# )

In [ ]:
layer_thick = []

In [ ]:
loss1 = {'Layer_4': 0.008160981325144135, 
'Layer_4_2': 0.00841587255241354, 
'Layer_5': 0.00893621835236717, 
'Layer_5_2': 0.011465427557568271, 
'Layer_0': 0.26086748791599207, 
'Layer_0_2': 0.2465395865656811, 
'Layer_2': 0.49013786668752085, 
'Layer_7': 0.2909012468006785, 
'Layer_7_2': 0.256640809321962, 
'Layer_9': 0.4564837506474886, 
'Layer_1': 0.35846329486246453, 
'Layer_1_2': 0.3416378031432378, 
'Layer_3': 0.5836236387395825, 
'Layer_6': 0.584839502470127, 
'Layer_8': 0.2951725992504724}

loss2 = {'Layer_5': 0.021325519083121983, 
         'Layer_5_2': 0.029311515030936916, 
         'Layer_6': 0.019134297952903195, 
         'Layer_6_2': 0.02997832220746138, 
         'Layer_1': 0.23591286317279858, 
         'Layer_1_2': 0.24644209232560482, 
         'Layer_3': 0.46840328395795633, 
         'Layer_8': 0.24384831882780136, 
         'Layer_8_2': 0.2590485051849109, 
         'Layer_10': 0.4456632668195654, 
         'Layer_0': 0.12454367379238514, 
         'Layer_2': 0.4416878983437709, 
         'Layer_4': 0.5690699914229048, 
         'Layer_7': 0.5561538271749439, 
         'Layer_9': 0.22851038721928338
         }
loss3 = {'Layer_4_1': 0.007869498363187635, 
         'Layer_4_2': 0.009136364030647841, 
         'Layer_5_1': 0.005090507014350036, 
         'Layer_5_2': 0.007690750162878706, 
         'Layer_0_1': 0.2622057105136101, 
         'Layer_0_2': 0.24586872888459188, 
         'Layer_2_1': 0.49359422772656986, 
         'Layer_7_1': 0.27701860790827837, 
         'Layer_7_2': 0.25648730382909524, 
         'Layer_9_1': 0.45189895360898963, 
         'Layer_1_1': 0.3717750760171587, 
         'Layer_1_2': 0.3325236463262538, 
         'Layer_3_1': 0.5953963162750471, 
         'Layer_6_1': 0.2968902016027462, 
         'Layer_6_2': 0.3081609669523285, 
         'Layer_8_1': 0.24620103178855923}
print(sum(loss1.values()))
print(sum(loss2.values()))
print(sum(loss3.values()))

4.2022860861927
3.919033762516349
4.167807891004293
